# Assignment 2
By Alexandra de Carvalho and Nuno Pedrosa

## Pre Processing

### Importing Modules

PySpark is imported, as an interface for Apache Spark in Python.

In [1]:
from pyspark import SparkContext
import pandas as pd
import ast

Tracks dataframe is created in order to filter tracks by the small subset.

### Filtering Dataset

In [17]:
df = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1])

COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),('track', 'genres'), ('track', 'genres_all')]
for column in COLUMNS:
    df[column] = df[column].map(ast.literal_eval)

COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),('album', 'date_created'), ('album', 'date_released'),('artist', 'date_created'), ('artist', 'active_year_begin'),('artist', 'active_year_end')]
for column in COLUMNS:
    df[column] = pd.to_datetime(df[column])

SUBSETS = ('small', 'medium', 'large')
df['set', 'subset'] = df['set', 'subset'].astype(pd.CategoricalDtype(categories=SUBSETS, ordered=True))

COLUMNS = [('track', 'genre_top'), ('track', 'license'),('album', 'type'), ('album', 'information'),('artist', 'bio')]
for column in COLUMNS:
    df[column] = df[column].astype('category')

Tracks dataframe is filtered to contain only the tracks belonging to the small subset.

In [18]:
small_tracks = df[df[('set','subset')] == 'small']
small_tracks

album                                                              \
         comments        date_created date_released        engineer favorites   
track_id                                                                        
2               0 2008-11-26 01:44:45    2009-01-05             NaN         4   
5               0 2008-11-26 01:44:45    2009-01-05             NaN         4   
10              0 2008-11-26 01:45:08    2008-02-06             NaN         4   
140             1 2008-11-26 01:49:59    2007-05-22             NaN         1   
141             0 2008-11-26 01:49:57    2009-01-16             NaN         1   
...           ...                 ...           ...             ...       ...   
154308          0 2017-03-05 04:57:38    2017-03-05             NaN         0   
154309          0 2017-03-05 04:57:38    2017-03-05             NaN         0   
154413          0 2017-03-07 18:44:11           NaT  Ernie Indradat         0   
154414          0 2017-03-07 18:44:11           NaT  Ernie Indradat         0   
155066          0 2017-03-24 19:40:34    2017-03-24        JBlanked         1   

                                                                            \
             id                                        information listens   
track_id                                                                     
2             1                                            <p></p>    6073   
5             1                                            <p></p>    6073   
10            6                                                NaN   47632   
140          61  <p>Alec K. Redfearn &amp; The Eyesores: Ellen ...    1300   
141          60  <p>A full ensamble of strings, drums, electron...    1304   
...         ...                                                ...     ...   
154308    22780                                                NaN   22334   
154309    22780                                                NaN   22334   
154413    22789  <p>A live performance at WFMU for Dark Night o...    3777   
154414    22789  <p>A live performance at WFMU for Dark Night o...    3777   
155066    22899  <p>Hip Hop, Old School type beat 2017, Trap Mu...    4524   

                                          \
                                producer   
track_id                                   
2                                    NaN   
5                                    NaN   
10                                   NaN   
140       Alec K. Refearn, Rob Pemberton   
141                                  NaN   
...                                  ...   
154308                           Fleslit   
154309                           Fleslit   
154413                     Julie Bennack   
154414                     Julie Bennack   
155066                          JBlanked   

                                                             ...       track  \
                                                       tags  ... information   
track_id                                                     ...               
2                                                        []  ...         NaN   
5                                                        []  ...         NaN   
10                                                       []  ...         NaN   
140                                                      []  ...         NaN   
141                                                      []  ...         NaN   
...                                                     ...  ...         ...   
154308    [fleslit, trap beat free use, trap beat free d...  ...         NaN   
154309    [fleslit, trap beat free use, trap beat free d...  ...         NaN   
154413                                                   []  ...         NaN   
154414                                                   []  ...         NaN   
155066    [old school beats, 2017 free instrumentals, fr...  ...         NaN   

                                 \
         in

The features dataframe is created, filtered by the small subset and then these rows are saved again to a file without header.

In [19]:
f = pd.read_csv('data/features.csv', index_col=0, header=[0, 1, 2])
f = f[f.index.isin(df.index)]
f.to_csv('data/small_processed_features.csv', header=False)

### Creating Context

A spark context is created, serving as the connection to a Spark cluster, further used to create and manipulate RDDs and broadcast variables on that cluster.

In [2]:
sc = SparkContext(appName="Assignment2")

22/05/17 15:39:51 WARN Utils: Your hostname, jarvis resolves to a loopback address: 127.0.1.1; using 192.168.32.72 instead (on interface wlp4s0)
22/05/17 15:39:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/17 15:39:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Ex 1.1

### Import the dataset

In this step, the file containing the dataset is read, with each file line as a rdd row.

In [20]:
data = sc.textFile('data/small_processed_features.csv')

In each row, each value is separated by a comma, so by splitting the text by comas, the row becomes a list of each row's values:
[track id, list of 518 features].

In [21]:
data = data.map(lambda line: line.split(","))

In [23]:
len(data.first())

519